In [ ]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import warnings 
import random
warnings.filterwarnings('ignore')

In [ ]:
shenzhen_grid = gpd.read_file('../data/shenzhen_grid/shenzhen_grid.shp')

In [ ]:
def convertStoT(coord_str):
    coord_str = coord_str.strip("()")

    coord_parts = coord_str.split(',')
    coord = (float(coord_parts[0]), float(coord_parts[1]))

    return coord

def draw_pentagram(ax, position, size=0.01, color='red'):
    # Points of a regular pentagon
    pentagon = np.array([[np.cos(2 * np.pi * i / 5), np.sin(2 * np.pi * i / 5)] for i in range(5)])
    pentagon = size * pentagon + position

    # Draw lines for the star
    for i in range(5):
        start = pentagon[i]
        end = pentagon[(i + 2) % 5]
        ax.plot([start[0], end[0]], [start[1], end[1]], color=color)

def drawNetworkOnMap(shenzhen_grid, csv_path, save_path,num_edges=1, alpha=0.2, node_size=20):
    data = pd.read_csv(csv_path, header=0)
    
    # Create a graph from the CSV data
    G = nx.Graph()
    for index, row in data.iterrows():
        coords_str = row[0]
        G.add_node(coords_str, pos=convertStoT(coords_str))

        sorted_indices = row[1:].argsort()[::-1]
        count = 0  
        for i in sorted_indices:
            # shift right 1
            connected_coords_str = data.columns[i+1]
            if row[i+1] != 0 and connected_coords_str != "no action":
                weight_value = row[i+1]
                G.add_edge(coords_str, connected_coords_str, weight=weight_value)
                count += 1
            if count == num_edges:
                break
    
    # Extract and scale weights for the edges
    weights = [G[u][v]['weight'] for u, v in G.edges()]
    max_absolute_weight = max(abs(w) for w in weights)
    normalized_weights = [(w + max_absolute_weight) / (2 * max_absolute_weight) for w in weights]
    scaled_weights = [5 * nw for nw in normalized_weights]
    pos = nx.get_node_attributes(G, 'pos')
    
    # Plot the network on the map
    f, ax = plt.subplots(1, 1, figsize=(20, 20))
    nx.draw_networkx_nodes(G, pos, ax=ax, node_size=node_size)
    nx.draw_networkx_edges(G, pos, width=scaled_weights, edge_color=scaled_weights, arrows=True, ax=ax)
    shenzhen_grid.plot(ax=ax, linewidth=1, alpha=alpha)

    draw_pentagram(ax, (114.03492647904385, 22.62162272422277))
    draw_pentagram(ax, (114.10926793627344, 22.53526575221365))
    plt.savefig(save_path)

In [ ]:
drawNetworkOnMap(shenzhen_grid, '../data/before_migrt_transProb.csv','../data/after_migrt/before_migrt.png', num_edges=1, alpha=0.2, node_size=20)

# After migration trans plot

In [ ]:
def drawNetworkAfterOnMap(shenzhen_grid, csv_path, save_path, before_coords_set, num_edges=1, alpha=0.2, node_size=20):
    data = pd.read_csv(csv_path, header=0)
    
    # Create a graph from the CSV data
    G = nx.Graph()
    for index, row in data.iterrows():
        coords_str = row[0]
        G.add_node(coords_str, pos=convertStoT(coords_str))
        if coords_str in before_coords_set:
            continue

        sorted_indices = row[1:].argsort()[::-1][:10]
        edges = []
        for i in sorted_indices:
            connected_coords_str = data.columns[i+1]
            if row[i+1] != 0 and connected_coords_str != "no action":
                weight_value = row[i+1]
                edges.append((connected_coords_str, weight_value))

        # If there are edges, randomly select one and add it
        if edges:
            selected_edge = random.choice(edges)
            G.add_edge(coords_str, selected_edge[0], weight=selected_edge[1])
    
    # Extract and scale weights for the edges
    weights = [G[u][v]['weight'] for u, v in G.edges()]
    max_absolute_weight = max(abs(w) for w in weights)
    normalized_weights = [(w + max_absolute_weight) / (2 * max_absolute_weight) for w in weights]
    scaled_weights = [5 * nw for nw in normalized_weights]
    pos = nx.get_node_attributes(G, 'pos')
    
    # Plot the network on the map
    f, ax = plt.subplots(1, 1, figsize=(20, 20))
    nx.draw_networkx_nodes(G, pos, ax=ax, node_size=node_size)
    nx.draw_networkx_edges(G, pos, width=scaled_weights, edge_color=scaled_weights, arrows=True, ax=ax)
    shenzhen_grid.plot(ax=ax, linewidth=1, alpha=alpha)

    draw_pentagram(ax, (114.03492647904385, 22.62162272422277))
    draw_pentagram(ax, (114.10926793627344, 22.53526575221365))
    plt.savefig(save_path)

In [ ]:
one_df=pd.read_csv('../data/one_travel_chain.csv')
one_df['origin_point'] = list(zip(one_df['lambda_o'],one_df['phi_o']))
one_df['destination_point'] = list(zip(one_df['lambda_d'],one_df['phi_d']))
df_before_migrt = one_df[one_df['date'] < one_df['migrt']]
df_after_migrt = one_df[one_df['date']>one_df['migrt']]

before_migrt_set,after_migrt_set = set(),set()
before_migrt_set.update(df_before_migrt['origin_point'].to_list())
before_migrt_set.update(df_before_migrt['destination_point'].tolist())
after_migrt_set.update(df_after_migrt['origin_point'].tolist())
after_migrt_set.update(df_after_migrt['destination_point'].tolist())

only_before = before_migrt_set - after_migrt_set

def tuple_to_string(coord_tuple):
    return f"({coord_tuple[0]}, {coord_tuple[1]})"

only_before_str = [tuple_to_string(coord) for coord in only_before]

In [ ]:
directory = '../data/after_migrt/transProb'
save_directory = '../data/after_migrt/img/'

if not os.path.exists(save_directory):
    os.makedirs(save_directory)
    
for f in tqdm(os.listdir(directory)):
    csv_path = os.path.join(directory, f)
    save_path = os.path.join(save_directory, f.replace('.csv', '.png'))
    drawNetworkAfterOnMap(shenzhen_grid, csv_path,save_path, only_before_str, num_edges=1, alpha=0.2, node_size=20)

In [ ]:
import imageio
import glob

file_names = [ '../data/after_migrt/before_migrt.png' ] + sorted(glob.glob('../data/after_migrt/img/*.png'))

images = [imageio.imread(file_name) for file_name in file_names]
imageio.mimsave('../img/cognitive_map.gif', images, duration=0.3) 